In [ ]:
import pathlib
import sqlite3
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

# ignore mix type warnings from pandas
import warnings

from pycytominer.cyto_utils.cells import SingleCells
from pycytominer import annotate, normalize, feature_select

warnings.filterwarnings("ignore")

In [ ]:
# Setting file paths
data_dir = pathlib.Path("../../").resolve(strict=True)
metadata_dir = pathlib.Path(
    "../../2021_09_01_VarChAMP-data/metadata/reprocessed/"
).resolve(strict=True)
out_dir = pathlib.Path(
    "../../2021_09_01_VarChAMP_local/6.downstream_analysis/results"
)
out_dir.mkdir(exist_ok=True)

# input file paths
plate = "2023-05-30_B1A1R1_P1T1"
plate_data = pathlib.Path(data_dir / "2023-05-30_B1A1R1_P1T1.sqlite").resolve(
    strict=True
)
plate_map = (metadata_dir / "2023_05_30_B1A1R1.csv").resolve(strict=True)

# setting output paths
sc_profiles_path = out_dir / "B1A1R1_P1T1_single_cell_profile.csv.gz"
sc_profiles_norm_path = (
    out_dir / "B1A1R1_P1T1_single_cell_profile_normalized.csv.gz"
)
sc_profiles_norm_select_path = (
    out_dir
    / "B1A1R1_P1T1_single_cell_profile_normalized_feature_selected.csv.gz"
)

In [ ]:
def is_feature_col(col):
    """Check if column is a feature"""
    return (
        col.startswith("Cell")
        or col.startswith("Cytoplasm")
        or col.startswith("Nuclei")
    )


def count(cur, table):
    """Count total number of rows for a table"""
    (num_rows,) = next(cur.execute(f"SELECT COUNT(*) FROM {table}"))
    return num_rows


def get_columns(cur, table):
    """Get feature and metadata columns lists"""
    ptr = cur.execute(f"SELECT * FROM {table} LIMIT 1")
    col_names = [obj[0] for obj in ptr.description]

    feat_cols = []
    meta_cols = []
    for col in col_names:
        if is_feature_col(col):
            feat_cols.append(col)
        else:
            meta_cols.append(col)

    return meta_cols, feat_cols


def load_compartment(cur, table):
    # Get data useful to pre-alloc memory
    num_cells = count(cur, table)
    meta_cols, feat_cols = get_columns(cur, table)
    num_meta, num_feats = len(meta_cols), len(feat_cols)

    feats = np.empty(shape=(num_cells, num_feats), dtype=np.float32)
    meta = pd.DataFrame(columns=meta_cols, index=range(num_cells))

    columns = ", ".join(meta_cols + feat_cols)
    query = f"SELECT {columns} from {table}"
    resultset = cur.execute(query)
    for i, row in tqdm(enumerate(resultset), total=num_cells):
        meta.loc[i] = row[:num_meta]
        feats[i] = row[num_meta:]
    return pd.concat(
        [meta, pd.DataFrame(columns=feat_cols, data=feats)], axis=1
    )

In [ ]:
# update compartment names and strata
strata = ["Metadata_Well", "Metadata_Plate"]
compartments = ["Cells", "Nuclei", "Cytoplasm"]

# Updating linking columns for merging all compartments
linking_cols = {
    "Cytoplasm": {
        "Cells": "Cytoplasm_Parent_Cells",
        "Nuclei": "Cytoplasm_Parent_Nuclei",
    },
    "Cells": {"Cytoplasm": "Cells_Number_Object_Number"},
    "Nuclei": {"Cytoplasm": "Nuclei_Number_Object_Number"},
}

merge_cols = ["ImageNumber", "TableNumber"]

In [ ]:
con = sqlite3.connect(plate_data)
cur = con.cursor()

In [ ]:
sc_df = ""
linking_check_cols = []
merge_suffix_rename = []

for left_compartment in linking_cols:
    for right_compartment in linking_cols[left_compartment]:
        # Make sure only one merge per combination occurs
        linking_check = "-".join(sorted([left_compartment, right_compartment]))
        if linking_check in linking_check_cols:
            continue

        # Specify how to indicate merge suffixes
        merge_suffix = [
            "_{comp_l}".format(comp_l=left_compartment),
            "_{comp_r}".format(comp_r=right_compartment),
        ]
        merge_suffix_rename += merge_suffix
        left_link_col = linking_cols[left_compartment][right_compartment]
        right_link_col = linking_cols[right_compartment][left_compartment]

        if isinstance(sc_df, str):
            sc_df = load_compartment(cur, left_compartment)

        sc_df = sc_df.merge(
            load_compartment(cur, right_compartment),
            left_on=merge_cols + [left_link_col],
            right_on=merge_cols + [right_link_col],
            suffixes=merge_suffix,
        )

        linking_check_cols.append(linking_check)

In [ ]:
rowdict = {
    "01": "A",
    "02": "B",
    "03": "C",
    "04": "D",
    "05": "E",
    "06": "F",
    "07": "G",
    "08": "H",
    "09": "I",
    "10": "J",
    "11": "K",
    "12": "L",
    "13": "M",
    "14": "N",
    "15": "O",
    "16": "P",
}

image_df = load_compartment(cur, "Image")
image_df = image_df[["TableNumber", "FileName_OrigAGP"]]
image_df["Metadata_Plate"] = plate
image_df["Metadata_Well"] = image_df["FileName_OrigAGP"].apply(
    lambda x: rowdict[x[1:3]] + x[4:6]
)
aligned_df = sc_df.merge(image_df, on=["TableNumber"], how="left")
aligned_df.drop(
    [
        "FileName_OrigAGP",
        "Cytoplasm_Parent_Cells",
        "Cytoplasm_Parent_Nuclei",
        "Cells_Number_Object_Number",
        "Nuclei_Number_Object_Number",
        "ObjectNumber_Cytoplasm",
        "ObjectNumber",
        "ObjectNumber_Cells",
        "TableNumber",
        "ImageNumber",
    ],
    inplace=True,
    axis=1,
)

meta_df = pd.read_csv(plate_map)
meta_df = meta_df[
    [
        "Metadata_Plate",
        "Metadata_Well",
        "node_type",
        "allele",
        "batch",
        "control",
        "Gene",
        "MT",
        "Variant",
        "Metadata_Sample_Unique",
        "Metadata_batch_Plate",
    ]
]

col_name = []
for i in meta_df.columns:
    if "Metadata" in i:
        col_name.append(i)
    else:
        col_name.append("Metadata_" + i)

meta_df = meta_df.set_axis(col_name, axis=1)

aligned_df = meta_df.merge(
    aligned_df, on=["Metadata_Plate", "Metadata_Well"], how="right"
)
aligned_df

In [ ]:
aligned_df.to_csv(sc_profiles_path, index=False, compression="gzip")

In [ ]:
# sql_query = """SELECT name FROM sqlite_master  
#   WHERE type='table';"""
# (num_rows,) = next(cur.execute('''SELECT COUNT(*) FROM Image'''))

### Normalization and Feature Selection

In [ ]:
# Whole plate normalization

normalize(
    profiles=sc_profiles_path,
    features="infer",
    image_features=False,
    meta_features="infer",
    samples="all",
    method='mad_robustize',
    output_file=sc_profiles_norm_path,
    compression_options="gzip",
)

print(f"Normalized profile saved in: {sc_profiles_norm_path}")

In [ ]:
# Perform plate level feature selection for now
feature_select(
    profiles=sc_profiles_norm_path,
    features="infer",
    image_features=True,
    samples="all",
    operation=["variance_threshold", "correlation_threshold", "blocklist", "drop_na_columns"],
    output_file=sc_profiles_norm_select_path,
    compression_options="gzip",
)

print(f"Feature selected profiles saved in: {sc_profiles_norm_select_path}")